In [1]:
#我们通过数据分析的得出了甜品店是最佳选择

In [2]:
'''
在qgis中做将上海划分成格网空间，结合python辅助做空间指标评价，得到餐饮选址位置。

'''

'\n在qgis中做将上海划分成格网空间，结合python辅助做空间指标评价，得到餐饮选址位置。\n\n'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['font.sans-serif']=['SimHei']
#⽤来正常显示中⽂标签
plt.rcParams['axes.unicode_minus']=False 
#⽤来正常显示负号 #有中⽂出现的情况，需要u'内容'

In [3]:
data = pd.read_excel('上海餐饮数据.xlsx')

In [4]:
data.head(5)

,类别,行政区,点评数,口味,环境,服务,人均消费,城市,Lng,Lat
0,烧烤,浦东新区,176,8.0,8.6,7.9,124,上海市,121.967860,30.884477
1,美食,闵行区,2,6.1,6.5,6.3,0,上海市,121.967781,30.883818
2,粤菜,浦东新区,141,6.7,7.2,6.6,141,上海市,121.933142,30.893224
3,海鲜,浦东新区,76,7.2,7.2,7.3,148,上海市,121.926062,30.899868
4,烧烤,浦东新区,600,7.2,7.6,7.0,143,上海市,121.925877,30.901100


In [7]:
df  =  data[data['类别']=='甜点']

In [9]:
df.to_csv('甜点.csv')

In [7]:
dealdata = pd.read_excel('处理后的数据.xlsx',sheet_name=0)

In [9]:
dealdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7222 entries, 0 to 7221
Data columns (total 7 columns):
wkt_geom    7222 non-null object
number      7222 non-null int64
length      7222 non-null float64
cy_count    2327 non-null float64
td_count    1119 non-null float64
lng         7222 non-null float64
lat         7222 non-null float64
dtypes: float64(5), int64(1), object(1)
memory usage: 395.1+ KB


In [10]:
#将空值填充上0

In [16]:
dealdata.fillna(0,inplace=True)

In [17]:
#指标统计 标准化

In [23]:
df2 = dealdata

In [24]:
df2.columns

Index(['wkt_geom', 'number', 'length', 'cy_count', 'td_count', 'lng', 'lat'], dtype='object')

In [25]:
#人口密度  道路密度 餐饮热度  同类竞品
#综合指标 = 人口密度指标*0.4 + 餐饮热度指标*0.3 + 道路密度指标*0.2 +同类竞品指标*0.1

In [34]:
df2['人口密度标准化'] =  (df2['number']-df2['number'].min()) / (df2['number'].max()-df2['number'].min())
df2['道路密度标准化'] =  (df2['length']-df2['length'].min()) / (df2['length'].max()-df2['length'].min())
df2['餐饮热度标准化'] =  (df2['cy_count']-df2['cy_count'].min()) / (df2['cy_count'].max()-df2['cy_count'].min())
df2['同类竞品标准化'] =  (df2['td_count'].max()-df2['td_count']) / (df2['td_count'].max()-df2['td_count'].min())

In [35]:
df2['综合指标'] = df2['人口密度标准化']*0.4 + df2['道路密度标准化']*0.3 +  df2['餐饮热度标准化']*0.2 + df2['同类竞品标准化']*0.1

In [38]:
df_finall = df2.sort_values('综合指标',ascending=False).reset_index()

In [41]:
df_finall.head(5)

,index,wkt_geom,number,length,cy_count,td_count,lng,lat,人口密度标准化,道路密度标准化,餐饮热度标准化,同类竞品标准化,综合指标
0,4372,Point (121.47311741266591412 31.23824359227818...,35036,15483.180214,1244.0,309.0,121.473117,31.238244,0.820688,0.528626,1.000000,0.000000,0.686863
1,4301,Point (121.45226984866562248 31.22897291834404...,36929,14010.454617,1049.0,237.0,121.452270,31.228973,0.865030,0.478344,0.843248,0.233010,0.681466
2,4826,Point (121.47210003973017933 31.30138099960866...,42691,15497.177842,149.0,39.0,121.472100,31.301381,1.000000,0.529104,0.119775,0.873786,0.670065
3,4441,Point (121.48347068197051613 31.24738756072670...,40623,14089.364209,362.0,41.0,121.483471,31.247388,0.951559,0.481038,0.290997,0.867314,0.669866
4,4232,Point (121.45241685328390702 31.21995331098462...,36929,12104.466337,972.0,188.0,121.452417,31.219953,0.865030,0.413270,0.781350,0.391586,0.665422


In [42]:
#制作空间散点图

In [43]:
#——————————————————————————————————

In [44]:
import warnings
warnings.filterwarnings('ignore')

from bokeh.plotting import figure,show,output_file
from bokeh.models import ColumnDataSource

from bokeh.models import HoverTool

In [47]:
'''
(3)绘图 制作空间散点图
'''
df_finall['size'] = df_finall['综合指标']*20
df_finall['final_score']= df_finall['综合指标']
df_finall['color'] = 'green'
df_finall['color'].iloc[:10] = 'red'

source = ColumnDataSource(df_finall)

output_file('project.html')

hover = HoverTool(tooltips=[('经度','@lng'),
                            ('纬度','@lat'),
                            ('最终得分','@final_score'),
                            ])

p = figure(plot_width = 800,plot_height = 800,
           title = '空间散点图',
           tools = [hover , 'box_select,reset,wheel_zoom,pan,crosshair'])

p.square(x = 'lng',y = 'lat',source = source,
        line_color = 'black',fill_alpha = 0.5,
        size = 'size',color = 'color')

show(p)